In [2]:
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
import torch
import torch.nn as nn
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import time
import psutil, os


class Wrapper:
    def __init__(self, config=None):
        self.config = config
    
    def regression_model_train(self, model, train_loader, val_loader, optimizer, criterion, scheduler):
        ground_truth = torch.empty((0, self.config.output_size), dtype=torch.float32, device=self.config.device)
        predicted = torch.empty((0, self.config.output_size), dtype=torch.float32, device=self.config.device)

        epochs = 30
        total_time = 0.0
        torch.cuda.reset_peak_memory_stats()

        for epoch in range(epochs):
            running_loss = 0.0
            count = 0
            start_time = time.time()
            for x, y in train_loader:
                if len(y.shape) == 1:
                    y = y.unsqueeze(-1)
                x, y = x.to(self.config.device), y.to(self.config.device)
                optimizer.zero_grad()
                output = model(x)
                loss = criterion(output, y)
                L2 = torch.sum((output - y)**2)
                # loss = loss + L2
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                count += 1
                if epoch == epochs - 1:
                    if self.config.model_type == 'seq2seq':
                        ground_truth = torch.cat((ground_truth, y[:, -1].unsqueeze(-1)), 0)
                        predicted = torch.cat((predicted, output[:, -1].unsqueeze(-1)), 0)
                    else:
                        ground_truth = torch.cat((ground_truth, y), 0)
                        predicted = torch.cat((predicted, output), 0)
            end_time = time.time()
            total_time += end_time - start_time
            val_loss = 0.0
            count = 0
            model.eval()
            with torch.no_grad():
                for x, y in val_loader:
                    if len(y.shape) == 1:
                        y = y.unsqueeze(-1)
                    x, y = x.to(self.config.device), y.to(self.config.device)
                    output = model(x)
                    loss = criterion(output, y)
                    L2 = torch.sum((output - y)**2)
                    # loss = loss + L2
                    val_loss += loss.item()
                    count += 1
            if scheduler is not None:
                scheduler.step(val_loss)
            print(f'Epoch [{epoch+1}/{self.config.epochs}], Loss: {running_loss / count}, Val Loss: {val_loss / count} Time: {end_time - start_time} s')
        
        print("🔁 Peak GPU memory used during 1 epoch (training):", torch.cuda.max_memory_allocated() / (1024 ** 2), "MB")
        print(f'Average time per epoch: {total_time / epochs} s')
        return ground_truth, predicted 
    
    def regression_model_test(self, model, test_loader):
        model.eval()
        ground_truth = torch.empty((0, self.config.output_size), dtype=torch.float32).to(self.config.device)
        predicted = torch.empty((0, self.config.output_size), dtype=torch.float32).to(self.config.device)

        process = psutil.Process(os.getpid())
        mem_before = process.memory_info().rss

        with torch.no_grad():
            for x, y in test_loader:
                if len(y.shape) == 1:
                    y = y.unsqueeze(-1)
                x, y = x.to(self.config.device), y.to(self.config.device)
                output = model(x)
                if self.config.model_type == 'seq2seq':
                    ground_truth = torch.cat((ground_truth, y[:, -1].unsqueeze(-1)), 0)
                    predicted = torch.cat((predicted, output[:, -1].unsqueeze(-1)), 0)
                else:
                    ground_truth = torch.cat((ground_truth, y), 0)
                    predicted = torch.cat((predicted, output), 0)

        
        mem_after = process.memory_info().rss
        print("🧠 PyTorch inference memory (CPU):", (mem_after - mem_before) / (1024 ** 2), "MB")
        return ground_truth, predicted 
    
    def classification_model_train(self, model, train_data, train_ground_truth, optimizer, criterion):
        epochs = self.config.epochs
        steps_per_epoch = train_data.shape[0] // self.config.batch_size

        ground_truth = torch.empty((0, self.config.output_size), dtype=torch.float32).to(self.config.device)
        predicted = torch.empty((0, self.config.output_size), dtype=torch.float32).to(self.config.device)
        epochs = 150
        
        for epoch in range(epochs):
            running_loss = 0.0
            count = 0

            for step in range(steps_per_epoch):
                inputs = train_data[step * self.config.batch_size : (step + 1) * self.config.batch_size]
                targets = train_ground_truth[step * self.config.batch_size : (step + 1) * self.config.batch_size]
                # inputs, targets = inputs.to(self.config.device), targets.to(self.config.device)  # Move data to GPU

                optimizer.zero_grad()
                output = model(inputs)

                loss = criterion(output, targets)
                loss.backward()
                optimizer.step()
                running_loss += loss.item()

                if epoch == epochs - 1:
                    ground_truth = torch.cat((ground_truth, targets), 0)
                    predicted = torch.cat((predicted, output), 0)

            print(f'Epoch [{epoch+1}/{self.config.epochs}], Loss: {running_loss / steps_per_epoch}')
        return ground_truth, predicted
        
    def classification_model_test(self, model, test_data, test_ground_truth):
        model.eval()
        ground_truth = torch.empty((0, self.config.output_size), dtype=torch.float32).to(self.config.device)
        predicted = torch.empty((0, self.config.output_size), dtype=torch.float32).to(self.config.device)

        step_per_epoch = test_data.shape[0] // self.config.batch_size
        for step in range(step_per_epoch):
            inputs = test_data[step * self.config.batch_size : (step + 1) * self.config.batch_size]
            targets = test_ground_truth[step * self.config.batch_size : (step + 1) * self.config.batch_size]
            # inputs, targets = inputs.to(self.config.device), targets.to(self.config.device)  # Move data to GPU

            output = model(inputs)
            ground_truth = torch.cat((ground_truth, targets), 0)
            predicted = torch.cat((predicted, output), 0)

        return ground_truth, predicted
    
    def imputation_model_train(self, model, train_loader, val_loader, optimizer, criterion, scheduler):
        ground_truth = torch.empty((0, self.config.output_size + 1), dtype=torch.float32, device=self.config.device)
        predicted = torch.empty((0, self.config.output_size), dtype=torch.float32, device=self.config.device)
        mask_list = torch.empty((0, self.config.output_size + 1), dtype=torch.float32, device=self.config.device)

        epochs = 50
        for epoch in range(epochs):
            running_loss = 0.0
            count = 0
            model.train()
            # loop over train loader and mask loader
            for x, y, mask in train_loader:
                x, y, mask = x.to(self.config.device), y.to(self.config.device), mask.to(self.config.device)
                optimizer.zero_grad()
                output = model(x)
                loss = criterion(output, y[:, :, 1:])
                L2 = torch.sum((output - y[:, :, 1:])**2)
                loss = loss
                loss.backward()
                optimizer.step()
                running_loss += loss.item()
                count += 1
                if epoch == epochs - 1:
                    ground_truth = torch.cat((ground_truth, y[:, -1, :]), 0)
                    predicted = torch.cat((predicted, output[:, -1, :]), 0)
                    mask_list = torch.cat((mask_list, mask[:, -1, :]), 0)

            val_loss = 0.0
            count = 0
            model.eval()
            with torch.no_grad():
                for x, y, mask in val_loader:
                    x, y = x.to(self.config.device), y.to(self.config.device)
                    output = model(x)
                    loss = criterion(output, y[:, :, 1:])
                    L2 = torch.sum((output - y[:, :, 1:])**2)
                    loss = loss
                    val_loss += loss.item()
                    count += 1
            if scheduler is not None:
                scheduler.step(val_loss)
            print(f'Epoch [{epoch+1}/{self.config.epochs}], Loss: {running_loss / count}, Val Loss: {val_loss / count}')
        return ground_truth, predicted, mask_list
    
    def imputation_model_test(self, model, test_loader):
        model.eval()
        ground_truth = torch.empty((0, self.config.output_size + 1), dtype=torch.float32).to(self.config.device)
        predicted = torch.empty((0, self.config.output_size), dtype=torch.float32).to(self.config.device)
        mask_list = torch.empty((0, self.config.output_size + 1), dtype=torch.float32).to(self.config.device)

        with torch.no_grad():
            for x, y, mask in test_loader:
                x, y = x.to(self.config.device), y.to(self.config.device)
                output = model(x)
                ground_truth = torch.cat((ground_truth, y[:, -1, :]), 0)
                predicted = torch.cat((predicted, output[:, -1, :]), 0)
                mask_list = torch.cat((mask_list, mask[:, -1, :]), 0)

        return ground_truth, predicted, mask_list

    def evaluation_metrics(self, ground_truth, predicted):
        # r2 = r2_score(ground_truth.cpu().detach().numpy(), predicted.cpu().detach().numpy())
        # mse = nn.MSELoss()
        # rmse = torch.sqrt(mse(ground_truth, predicted))
        #mae using numpy
        mae_loss_value = np.abs(ground_truth - predicted).mean()
        # mae_loss_value = mae_loss(ground_truth, predicted)

        num = np.abs(ground_truth - predicted).sum(axis=0)
        den = 2*ground_truth.sum(axis=0)
        eac = (1 - num/den)

        nde = np.sum((ground_truth - predicted) ** 2) / np.sum((ground_truth ** 2), axis=0) 

        return mae_loss_value, eac, nde
    
    def calculate_metrics(self, pred, ture):
        """
        :param pred: numpy
        :param ture: numpy
        :return: None
        """
        print("F1-score", f1_score(pred, ture))
        print("Acc", accuracy_score(pred, ture))
        print("Recall", recall_score(pred, ture))
        print("Precision", precision_score(pred, ture))

    def appliance_wise_metrics(self, appliance_list, ground_truth, predicted):
        #iterate the list of appliances
        print(appliance_list)
        for i in appliance_list['appliances']:
            print(f"""Appliance {i["name"]}""")
            mae, eac, nde = self.evaluation_metrics(ground_truth[:, i["id"]], predicted[:, i["id"]])
            print(f"MAE: {mae}, EAC: {eac}, NDE: {nde}")
